In [1]:
import os
import torch
from unsloth import FastLanguageModel, is_bfloat16_supported
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset
from prompts import prompt, system

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [2]:
cls2name = {
    "FC": "Fully compliant",
    "LC": "Largely compliant",
    "PC": "Partially compliant",
    "NC": "Not compliant"
}

In [3]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="mistralai/Mistral-Nemo-Instruct-2407",
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
    # token="hf_..."
)

==((====))==  Unsloth 2024.9.post4: Fast Mistral patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla V100-PCIE-32GB. Max memory: 31.739 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.0+cu121. CUDA = 7.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.24+cu118. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 2/2 [04:28<00:00, 134.37s/it]


In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=128,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=56,
    use_rslora=False,
    loftq_config=None
)

Unsloth 2024.9.post4 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


In [5]:
EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    texts = []
    for task, name, uc_text, ssts_text, answer in zip(examples["task"], examples["name"], examples["uc_text"], examples["ssts_text"], examples["answer"]):
        if task == "Complience Level":
            answer = cls2name[answer]
        text = prompt.format(system[task], name, uc_text, ssts_text, task, answer) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

In [6]:
data_files = {"train": "./train data/dataset.csv"}
dataset = load_dataset("csv", data_files=data_files)
dataset = dataset.map(formatting_prompts_func, batched=True)

Map: 100%|██████████| 33/33 [00:00<00:00, 581.20 examples/s]


In [7]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset["train"],
    dataset_text_field="text",
    max_seq_length=2048,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        num_train_epochs=6,
        per_device_train_batch_size=1, # т.к. мало данных
        gradient_accumulation_steps=1,
        warmup_steps=10,
        learning_rate=5e-5,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim= "adamw_8bit",
        weight_decay=0.0025,
        lr_scheduler_type="linear",
        seed=56,
        output_dir="outputs",
        load_best_model_at_end=False,
        greater_is_better=False,
        eval_steps=dataset["train"].num_rows,
        save_steps=dataset["train"].num_rows,
        eval_strategy="no",
        save_strategy="steps"
    )
)

Map (num_proc=2): 100%|██████████| 33/33 [00:01<00:00, 18.68 examples/s]
/home/jupyter/.local/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:401: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(


In [8]:
trainer_stats = trainer.train()
model = trainer.model

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 33 | Num Epochs = 6
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 1
\        /    Total batch size = 1 | Total steps = 198
 "-____-"     Number of trainable parameters = 57,016,320
  1%|          | 1/198 [00:17<56:34, 17.23s/it]

{'loss': 2.1397, 'grad_norm': 6.9088544845581055, 'learning_rate': 5e-06, 'epoch': 0.03}


  1%|          | 2/198 [00:18<26:10,  8.01s/it]

{'loss': 2.4268, 'grad_norm': 10.900650978088379, 'learning_rate': 1e-05, 'epoch': 0.06}


  2%|▏         | 3/198 [00:20<16:47,  5.17s/it]

...

{'loss': 0.0066, 'grad_norm': 0.46007052063941956, 'learning_rate': 2.6595744680851066e-07, 'epoch': 5.97}


100%|██████████| 198/198 [05:54<00:00,  1.67s/it]

{'loss': 0.0039, 'grad_norm': 0.29465770721435547, 'learning_rate': 0.0, 'epoch': 6.0}


100%|██████████| 198/198 [05:57<00:00,  1.81s/it]

{'train_runtime': 357.4326, 'train_samples_per_second': 0.554, 'train_steps_per_second': 0.554, 'train_loss': 0.33379795959553293, 'epoch': 6.0}


In [9]:
model.save_pretrained("./Mistral-finetuned")
tokenizer.save_pretrained("./Mistral-finetuned")

('./Mistral-finetuned/tokenizer_config.json',
 './Mistral-finetuned/special_tokens_map.json',
 './Mistral-finetuned/tokenizer.json')